## Functions

In [22]:
#package loaded
#pip install PySimpleGUI
#pip install pyperclip
#pip install chess

from skimage import io, transform
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [23]:
# global vars
#g_dataset_dir = "../dataset/" # when OS system is used
print("start reading!")
g_dataset_dir = "./dataset/"   # when window system is used
g_train_dir = g_dataset_dir + "/train/"
g_test_dir = g_dataset_dir + "/test/"
print("read successfully!")
g_image_size = 400
#期盘长度
g_grid_row = 8
g_grid_col = 8

g_grid_num = g_grid_row * g_grid_col
#旗子格子数量
g_grid_size = int(g_image_size / g_grid_row)
#格子的长度

#Processing 1 - scale down 
g_down_sampled_size = 200
g_down_sampled_grid_size = int(g_grid_size / (g_image_size / g_down_sampled_size))

# global instance of mapping of char vs chess pieces
# reference: Forsyth–Edwards Notation, https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation
# 
# pawn = "P", knight = "N", bishop = "B", rook = "R", queen = "Q" and king = "K"
# White pieces are designated using upper-case letters ("PNBRQK") while black pieces use lowercase ("pnbrqk")
# we use 0 to note an empty grid.
# 13 items in total.

g_piece_mapping = {
    "P" : "pawn",
    "N" : "knight",
    "B" : "bishop",
    "R" : "rook",
    "Q" : "queen",
    "K" : "king",

    "p" : "pawn",
    "n" : "knight",
    "b" : "bishop",
    "r" : "rook",
    "q" : "queen",
    "k" : "king",

    "0" : "empty_grid"
}

g_num_labels = len(g_piece_mapping)
g_num_labels

start reading!
read successfully!


13

In [24]:
#DataHelper.py

import os

import cv2
from skimage import io
import numpy as np

import glob
import h5py

# get clean name by a path, where in our case this gets the FEN conviniently
def GetCleanNameByPath(file_name):
    return os.path.splitext(os.path.basename(file_name))[0]

# get full paths to the files in a directory.
def GetFileNamesInDir(path_name, extension="*", num_return = 0):
    print("loading")
    if num_return == 0:
        return glob.glob(path_name + "/*." + extension)
    else:
        return glob.glob(path_name + "/*." + extension)[:num_return]

# get name list
def GetCleanNamesInDir(path_name, extension = "*", num_return = 0):
    names = GetFileNamesInDir(path_name, extension)
    offset = len(extension) + 1
    clean_names = [os.path.basename(x)[:-offset] for x in names]
    if num_return == 0:
        return clean_names
    else:
        return clean_names[:num_return]

# read dataset
def ReadImages(file_names, path = "", format = cv2.IMREAD_COLOR):
    if path == "":
        return [cv2.imread(f, format) for f in file_names]
    else:
        return [cv2.imread(path + "/" + f, format) for f in file_names]

# read image by name
def ReadImage(file_name):
    return io.imread(file_name)

print("functions loaded")

functions loaded


In [25]:
#BoardHelper.py  

import re
import string
from collections import OrderedDict 

import numpy as np
import skimage.util
from skimage.util.shape import view_as_blocks


# from ChessGlobalDefs import * - global definitions

#FEN TO LABELS OF SQUARES
def FENtoL(fen): 
    rules = {
        r"-": r"",
        r"1": r"0",
        r"2": r"00",
        r"3": r"000",
        r"4": r"0000",
        r"5": r"00000",
        r"6": r"000000",
        r"7": r"0000000",
        r"8": r"00000000",
    }

    for key in rules.keys():
        fen = re.sub(key, rules[key], fen)

    return list(fen)


# Label array to char list:
def LabelArrayToL(arr):
    rules = {
        0 : "P",
        1 : "N",
        2 : "B",
        3 : "R",
        4 : "Q",
        5 : "K",

        6 : "p",
        7 : "n",
        8 : "b",
        9 : "r",
       10 : "q",
       11 : "k",

       12 : "0"
    }

    flattened = arr.flatten(order = "C")

    L = []

    for x in flattened:
        L.append(rules[x])

    return L

# char list to FEN
def LtoFEN(L):

    FEN = ""
    
    for y in range(8):
        counter = 0
        for x in range(8):
            idx = x + y * 8
            char = L[idx]

            if char == "0":
                counter += 1
                if x == 7:
                    FEN += str(counter)
            else:
                if counter:
                    FEN += str(counter)
                    counter = 0

                FEN += char
        if y != 7:
            FEN += "-"
        
            
    return FEN



# FEN to one-hot encoding, in our case, it returns an 64 by 13 array, with each row as a one-hot to a grid.
def FENtoOneHot(fen):

    # this rule is in the same format as g_piece_mapping
    #rules = {
    #    "P" : np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    #    "N" : np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    #    "B" : np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    #    "R" : np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    #    "Q" : np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
    #    "K" : np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
    #                    
    #    "p" : np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
    #    "n" : np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
    #    "b" : np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
    #    "r" : np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
    #    "q" : np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
    #    "k" : np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
    #                    
    #    "0" : np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
    #}
    rules = {
        "P" : 0,
        "N" : 1,
        "B" : 2,
        "R" : 3,
        "Q" : 4,
        "K" : 5,

        "p" : 6,
        "n" : 7,
        "b" : 8,
        "r" : 9,
        "q" : 10,
        "k" : 11,

        "0" : 12
    }

    L = FENtoL(fen)
    one_hot_array = np.zeros((g_grid_num, g_num_labels), dtype = np.int32) # 64 by 13
    for i, c in enumerate(L):
        one_hot_array[i, rules[c]] = 1

    return one_hot_array

# get 8*8 char matrix
def LtoCharMat(l):
    if type(l) == list:
        return np.array(l).reshape((8,8))
    if type(l) == str:
        return np.array([l]).reshape((8,8))

def GetBoardCell(board_image, row = 0, col = 0, size = 50):
    return np.array(board_image)[row*size:(row+1)*size,col*size:(col+1)*size]

# get grids of image
def ImageToGrids(image, grid_size_x, grid_size_y):
    return skimage.util.shape.view_as_blocks(image, block_shape = (grid_size_y, grid_size_x, 3)).squeeze(axis = 2)
print("functions loaded")

functions loaded


## Functions- new

### Confusion matrix - heatmap

In [ ]:
from sklearn.metrics import confusion_matrix

# function - to create confusion matrix
def conf_mat(true,pred):
    A = pd.Series(true, name='Actual')
    P = pd.Series(pred, name='Predict')
    #conf_matt  = pd.crosstab(A, P, margins=True)
    conf_matt  = pd.crosstab(A, P)
    return conf_matt


#function - to create heatmap from confusion matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(conf_matt, cmap=plt.cm.gray_r): 
    conf_norm = conf_matt / conf_matt.sum(axis=1)  
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(conf_norm, cmap=cmap)
    fig.colorbar(cax)
    tick_marks = np.arange(len(conf_norm.columns))
    plt.xticks(tick_marks, conf_norm.columns)
    plt.yticks(tick_marks, conf_norm.index)
    plt.ylabel(conf_norm.index.name)
    plt.xlabel(conf_norm.columns.name)
print("functions loaded")

### image and label preprocessing to df

In [ ]:
# modified function 

def PreprocessImage_df(image):
        image = transform.resize(image, (g_down_sampled_size, g_down_sampled_size), mode='constant')
        
        # 1st and 2nd dim is 8
        grids = ImageToGrids(image, g_down_sampled_grid_size, g_down_sampled_grid_size)
        g = grids.reshape(g_grid_row * g_grid_col, g_down_sampled_grid_size, g_down_sampled_grid_size, 3)
        df =pd.DataFrame()
        for i in range(8):
            for j in range(8):
                df = df.append(pd.Series(g[i][j].reshape(75)), ignore_index=True)
        return df

In [ ]:
# modifed function 

def func_generator_df(train_file_names):
    df =pd.DataFrame()
    for image_file_name in train_file_names:
        img = ReadImage(image_file_name) 
        x = PreprocessImage_df(img) # 64 x 75 dataframe
        y = np.array(FENtoL(GetCleanNameByPath(image_file_name)))
        x['y'] = y
        df = df.append(x, ignore_index=True)
    return df

In [ ]:
from sklearn.model_selection import train_test_split
print("start loading")
num_train = 800
train_file_names = GetFileNamesInDir(g_train_dir, extension = "jpeg",num_return = num_train)
# train image path list
df= func_generator_df(train_file_names)
X = df.drop('y', axis=1)
Y = df['y']
print("loaded")

In [ ]:
def show_imgs(n_rows,n_cols,x_data,y_data,class_name):
    assert len(x_data)==len(y_data)
    plt.figure(figsize=(n_cols*1.4,n_rows*1.6))
    for row in range(n_rows):
        for col in range(n_cols):
            index=n_cols*row+col
            plt.subplot(n_rows,n_cols,index+1)
            plt.imshow(x_data[index].reshape(25,25),cmap="binary",interpolation='nearest')
            plt.axis('off')
            plt.title(class_name[y_data[index]])
            
    plt.show()

### split data to train and validation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.33, random_state=0)

In [ ]:
# SVC

In [21]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
print('strat')
# clf=svm.SVC(gamma='scale',decision_function_shape='ovo')
svc = svm.SVC()
# clf = GridSearchCV(svc, { 'kernel':('linear', 'rbf'),'C':[1:2]})
clf = GridSearchCV(svc, {'kernel':('linear', 'rbf'), 'C':[ 1]})
# clf=svm.SVC(gamma='scale')
clf.fit(X_train,Y_train)
print('finished')

strat


KeyboardInterrupt: 

In [21]:
# Print grid search results
from sklearn.metrics import classification_report

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

print('Grid search mean and stdev:\n')

for mean, std, param in zip(means, stds, params):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, param))

# Print best params
print('\nBest parameters:', clf.best_params_)

print("Detailed classification report:")
print()
print(classification_report(Y_val, clf.predict(X_val)))
print()

Grid search mean and stdev:

0.807 (+/-0.001) for {'C': 1, 'kernel': 'linear'}
0.807 (+/-0.001) for {'C': 1, 'kernel': 'rbf'}
0.807 (+/-0.001) for {'C': 2, 'kernel': 'linear'}
0.807 (+/-0.001) for {'C': 2, 'kernel': 'rbf'}

Best parameters: {'C': 1, 'kernel': 'linear'}
Detailed classification report:

              precision    recall  f1-score   support

           0       0.81      1.00      0.90      1374
           B       0.00      0.00      0.00        42
           K       0.00      0.00      0.00        30
           N       0.00      0.00      0.00        22
           P       0.00      0.00      0.00        26
           Q       0.00      0.00      0.00        12
           R       0.00      0.00      0.00        26
           b       0.00      0.00      0.00        48
           k       0.00      0.00      0.00        21
           n       0.00      0.00      0.00        24
           p       0.00      0.00      0.00        27
           q       0.00      0.00      0.00     

c:\users\jin-d\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:

def getAccuracy(y_test,y_pre):
    res=0
    for k,v in zip(y_test,y_pre):
        if k==v:
            res+=1
 
    return res/y_test.shape[0]
 
y_pred=clf.predict(X_val)
print("acc:",getAccuracy(y_pred,Y_val))

acc: 0.8130177514792899
